In [15]:
import pandas as pd
import geopandas as gpd
from shapely import MultiPoint
import itertools
import os
import shutil
import zipfile
import subprocess

In [ ]:
milan_buildings_polys = gpd.read_file("data/footprints/Milano.shp").geometry
milan_buildings_points = [building.convex_hull.exterior.coords for building in milan_buildings_polys]
outer_milan = MultiPoint(list(itertools.chain.from_iterable(milan_buildings_points))).convex_hull.buffer(-1e-9)
gpd.GeoDataFrame(pd.DataFrame([{"id": "0"}]), geometry=[outer_milan],crs="EPSG:7791").to_file("data/coverage.shp")
outer_milan

In [ ]:
sample_input_gdf = gpd.read_file("data/coverage.shp").to_crs("EPSG:4326")
sample_coverage = list(sample_input_gdf["geometry"])[0]
CONTRACT_NUMBER = 145

# Specify intersection
contract_data = gpd.read_file(f"metadata/contract_{CONTRACT_NUMBER}/metadata_contract_{CONTRACT_NUMBER}.shp")

def get_intersection_tiles(input_polygon, lidar_coverage):
    intersection_array = lidar_coverage.geometry.map(lambda x: x.intersects(input_polygon))
    return lidar_coverage[intersection_array][["id", "region"]]

intersection = get_intersection_tiles(sample_coverage, contract_data)

# Copy files from HDD
source_path_prefix = "/Volumes/Seagate Expansion Drive/just_points"
destination_path_prefix = "data/lidar_points/raw"

def get_point_file_path(region, contract, filename):
    return os.path.join(source_path_prefix, region, f"Contratto_{contract}", "PUNTI", f"{filename}.zip")

files_to_copy = [get_point_file_path(row["region"], str(CONTRACT_NUMBER), row["id"]) for _, row in intersection.iterrows()]
for filepath in files_to_copy:
    dst = os.path.join(destination_path_prefix, filepath.split("/")[-1])
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copy(filepath, dst)

# Extract Zips
for filename in os.listdir(destination_path_prefix):  
    filepath = os.path.join(destination_path_prefix, filename)
    if zipfile.is_zipfile(filepath):
        with zipfile.ZipFile(filepath) as item:
           item.extractall(destination_path_prefix) 

# Remove Zips
for filename in os.listdir(destination_path_prefix): 
    if filename.endswith("zip"):
        os.remove(os.path.join(destination_path_prefix, filename))


In [17]:
footprints_raw = gpd.read_file("data/footprints/Milano.shp").to_crs("EPSG:4326")
coverage_140 = gpd.read_file("metadata/contract_140/metadata_contract_140.shp")
coverage_145 = gpd.read_file("metadata/contract_145/metadata_contract_145.shp")

def prepare_points(points_filename):
    points = pd.read_csv(f"data/lidar_points/{points_filename}.xyz", names=["long","lat","elevation","_1","_2"], delim_whitespace=True)
    points = points[["long","lat", "elevation"]].rename(columns={'lat': 'x', 'long': 'y', 'elevation': 'z'})
    points.insert(0, 'id', range(0, len(points)))
    path = f"data/tmp/points_{points_filename}.csv"
    points.to_csv(path, index=False)
    return path

def prepare_footprints(points_filename):
    coverage_tiles_140 = list(coverage_140[coverage_140["id"] == points_filename].geometry)
    coverage_tiles_145 = list(coverage_145[coverage_145["id"] == points_filename].geometry)
    
    coverage_tile = coverage_tiles_140[0] if coverage_tiles_140 else coverage_tiles_145[0]
    footprints_intersection = footprints_raw.geometry.map(lambda x: x.intersects(coverage_tile))
    footprints = footprints_raw[footprints_intersection][["OBJECTID", "edifc_stat", "edifc_ty", "edifc_uso", "geometry"]]\
        .rename(columns={'OBJECTID': 'id'})
    path = f"data/tmp/footprints_{points_filename}.csv"
    footprints.to_csv(path, index=False)
    return path

In [20]:
#for filename in os.listdir("data/lidar_points/"):
for filename in ['D45420917_0101_Punti.xyz']:
    points_filename = filename[:-4]
    print(points_filename)
    points_path = prepare_points(points_filename)
    footprints_path = prepare_footprints(points_filename)
    output_path = f"data/postgis_output/{points_filename}.csv"
    os.system(f"./postgis_processing.sh {points_path} {footprints_path} {output_path}")
    os.remove(footprints_path)
    os.remove(points_path)

D45420917_0101_Punti
1e4d22617eac5881dd36bff6b35a59ad1a04c7ba0232029545031c54a2710f31
CREATE TABLE
CREATE TABLE
COPY 1679045
UPDATE 1679045
COPY 283
UPDATE 283
UPDATE 283
COPY 283
skynet
skynet
